In [ ]:
from fastai.vision.all import *
from fastai.data.all import *

In [ ]:
DATASET_NAME = 'MAGNETIC_TILE_SURFACE_DEFECTS'
CODE_NAMES = [0, 1]

PROJ_PATH = Path.cwd().parent
DATASET_PATH = PROJ_PATH/'data'/DATASET_NAME
print(PROJ_PATH)
print(DATASET_PATH)


In [ ]:
fpaths = get_image_files(DATASET_PATH/'images')
fpaths

In [ ]:
def get_mask_path(img_path): 
    msk_path = img_path.parent.parent/'masks'/f'{img_path.stem}.png'
    return msk_path

get_mask_path(fpaths[0])

In [ ]:
img = Image.open(fpaths[0])
img.size

In [ ]:
if min(img.size) >= 512:
    sz = 512
elif min(img.size) < 100:
    import warnings
    warnings.warn(f"Image size of {img.size} is very small")
    sz = min(img.size)
else:
    sz = min(img.size)

item_tfms_sz = sz
batch_tfms_sz = sz
item_tfms_sz, batch_tfms_sz

In [ ]:
dls = SegmentationDataLoaders.from_label_func(
    path=DATASET_PATH, 
    fnames=fpaths,
    label_func=get_mask_path,
    codes=CODE_NAMES,
    bs=16, 
    splitter=RandomSplitter(valid_pct=0.2, seed=0),
    item_tfms = Resize(item_tfms_sz),
    batch_tfms =[*aug_transforms(size=batch_tfms_sz), 
                 Normalize.from_stats(*imagenet_stats)]
)

#dls.device = 'cpu' # uncomment to use CPU, might take longer to train

dls.show_batch()
batch = dls.one_batch()
print(batch[0].min(), batch[0].max())
print(batch[1].min(), batch[1].max())

In [ ]:
len(dls.train_ds), len(dls.valid_ds), len(dls.train_ds) + len(dls.valid_ds)

In [ ]:
metrics = [DiceMulti]  

In [ ]:
pickle_fpath = DATASET_PATH/'export_learn_obj.pkl'
learn = unet_learner(dls, resnet34, metrics=metrics)

learn.fine_tune(15)
learn.export(fname=pickle_fpath)

In [ ]:
learn.show_results()

In [ ]:
loss_val, dice_val = learn.validate()
loss_val, dice_val

In [ ]:
interp = SegmentationInterpretation.from_learner(learn)
interp.plot_top_losses(10,figsize=(10,40))